In [42]:

# def read_data(file_path):
#     with open(file_path, 'r') as file:
#         data = [row.split('\t') for row in file.read().split('\n')]
#         header = data[0]
#         data = [[float(val) if i > 0 else val for i, val in enumerate(row)] for row in data[1:-1]]
#     return header, data

# def extract_features_target(data):
#     features = [row[1:-1] for row in data]  # Exclude the ID column and the last column (target)
#     target = [row[-1] for row in data]  # Extract the last column (target)
#     return features, target

# def split_data(features, target, test_size=0.2, random_state=None):
#     num_samples = len(features)
#     num_test_samples = int(test_size * num_samples)  # Fix: Change 'test_ratio' to 'test_size'
    
#     if random_state is not None:
#         random.seed(random_state)
#         indices = random.sample(range(num_samples), num_test_samples)
#     else:
#         indices = range(num_test_samples)
    
#     test_features = [features[i] for i in indices]
#     test_target = [target[i] for i in indices]
    
#     train_features = [features[i] for i in range(num_samples) if i not in indices]
#     train_target = [target[i] for i in range(num_samples) if i not in indices]
    
#     return train_features, train_target, test_features, test_target


# #  Training the Model
# def train_linear_regression(features, target):
#     num_samples = len(features)
    
#     if num_samples == 0:
#         raise ValueError("No samples found. Please check your data.")
    
#     num_features = len(features[0])
    
#     if num_features == 0:
#         raise ValueError("No features found. Please check your data.")
    
#     # Initialize weights and bias
#     weights = [0.0] * num_features
#     bias = 0.0
    
#     # Learning rate
#     learning_rate = 0.0001
    
#     # Number of iterations (epochs)
#     num_epochs = 1000
    
#     for epoch in range(num_epochs):
#         error_sum = 0.0
#         for i in range(num_samples):
#             prediction = sum(weights[j] * features[i][j] for j in range(num_features)) + bias
#             error = target[i] - prediction
            
#             for j in range(num_features):
#                 weights[j] += learning_rate * error * features[i][j]
#             bias += learning_rate * error
            
#             error_sum += error ** 2
        
#         # Print mean squared error for this epoch
#         print(f'Epoch {epoch + 1}: Mean Squared Error = {error_sum / num_samples}')
    
#     return weights, bias



# # Making Predictions
# def predict(features, weights, bias):
#     predictions = []
#     for feature in features:
#         prediction = sum(weights[j] * feature[j] for j in range(len(feature))) + bias
#         predictions.append(prediction)
#     return predictions

# #  Evaluating the Model
# def mean_squared_error(predictions, actual):
#     return sum((p - a) ** 2 for p, a in zip(predictions, actual)) / len(predictions)

# # Usage Example
# data = read_data("handwriting_alzheimers.csv")
# features, target = extract_features_target(data)
# train_features, train_target, test_features, test_target = split_data(features, target)

# weights, bias = train_linear_regression(train_features, train_target)
# predictions = predict(test_features, weights, bias)
# mse = mean_squared_error(predictions, test_target)
# print(f'Mean Squared Error on Test Data: {mse}')

In [46]:
import csv
import numpy as np 
import matplotlib.pyplot as plt
import os
import itertools
import pandas as pd
import seaborn as sns
import matplotlib.pyplot
import numpy.linalg 
import numpy.random

def parseData(filename):
    csv_data = pd.read_csv(filename)
    print(csv_data.describe())
    numpy_data = csv_data.values
    rows, columns = numpy_data.shape
    X = numpy_data[:, :columns - 1]
    y = numpy_data[:, columns - 1:]
    X = np.array(X)
    y = np.array(y)
    return X, y

def splitData(X, y, trainSplit, valSplit, testSplit):
    trainStop = int(trainSplit * X.shape[0])
    valStop = int((trainSplit + valSplit) * X.shape[0])
    train_x = X[0:trainStop, :]
    train_y = y[0:trainStop]
    val_x = X[trainStop:valStop, :]
    val_y = y[trainStop:valStop]
    test_x = X[valStop:, :]
    test_y = y[valStop:]
    return train_x, train_y, val_x, val_y, test_x, test_y


def normalize(X):
    rangeX = np.zeros(X.shape[1])
    minX = np.zeros(X.shape[1])
    normX = np.zeros(X.shape)

    for i in range(X.shape[1]):
        minX[i] = min(X[:, i])
        rangeX[i] = max(X[:, i]) - minX[i]
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            normX[i][j] = (X[i][j] - minX[j]) / rangeX[j]

    return normX


class LinearRegression:
    def __init__(self):
        self.coefficients = None

    def fit(self, X, y):
        # Add a column of ones to the input data for the bias term
        X = np.column_stack((np.ones(X.shape[0]), X))
        
        # Calculate coefficients using the closed-form solution
        X_transpose = X.T
        self.coefficients = np.linalg.inv(X_transpose.dot(X)).dot(X_transpose).dot(y)

    def predict(self, X):
        # Add a column of ones to the input data for the bias term
        X = np.column_stack((np.ones(X.shape[0]), X))
        
        # Predict using the learned coefficients
        y_pred = X.dot(self.coefficients)
        return y_pred

    def mean_squared_error(self, y_true, y_pred):
        # Calculate the mean squared error
        mse = ((y_true - y_pred) ** 2).mean()
        return mse
    
    
    def rss(self, y_true, y_pred):
        # Calculate the Residual Sum of Squares (RSS)
        rss = ((y_true - y_pred) ** 2).sum()
        return rss

# Sample usage:
if __name__ == "__main__":
    
    X,y=parseData("handwriting_alzheimers.csv")
    X = X[:, 1:]
    y = np.where(y == "P", 1, y)
    y = np.where(y == "H", -1, y)
    X = normalize(X)
#     X, y = shuffle(X, y)
#     X, y = shuffle(X, y)
#     X, y = shuffle(X, y)
    
    # Initialize the LinearRegression model
    model = LinearRegression()
    
    # Fit the model on the training data
    model.fit(X, y)
    
    # Predict on new data
    y_pred = model.predict(X)
    
    # Calculate mean squared error
    mse = model.mean_squared_error(y, y_pred)
    
    # Print the coefficients and mean squared error
    print("Coefficients:", model.coefficients)
    print("Mean Squared Error:", mse)
    
    
    # Calculate RSS
    rss = model.rss(y, y_pred)
    
    # Print the RSS
    print("Residual Sum of Squares (RSS):", rss)



Coefficients: [[-27365960673877.094]
 [1418404649242942.5]
 [106.7034912109375]
 [521383364810.3556]
 [382152255900.05054]
 [586.5325393676758]
 [204.91064453125]
 [-591.3680419921875]
 [1302.5440979003906]
 [-727987345276.7151]
 [541.513916015625]
 [-673.107421875]
 [511.49871826171875]
 [-946.162109375]
 [-159.68756103515625]
 [584849924299646.0]
 [293.29076766967773]
 [323.066650390625]
 [-1982346333944888.5]
 [3444927337211463.5]
 [-235.16717529296875]
 [61820071287.49432]
 [33766362503.08664]
 [-10.179519653320312]
 [-30.934066772460938]
 [-3634.32568359375]
 [-131.22079467773438]
 [-71459755381.29523]
 [3650.99658203125]
 [728.0029907226562]
 [14.2904052734375]
 [330.81866455078125]
 [1168.3699035644531]
 [1606390825715484.2]
 [-312.5876178741455]
 [-336.8681640625]
 [-4929334203036855.0]
 [-743307995427564.2]
 [158.05322265625]
 [200024173419.72906]
 [91740568832.09949]
 [-226.99755859375]
 [-285.307373046875]
 [1148.7689819335938]
 [-581.6065673828125]
 [-247313630908.21765]
 [

In [45]:
class Perceptron:
    def __init__(self, learning_rate=0.01, n_iterations=1000):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations

    def fit(self, X, y):
        self.X = X
        self.y = y
        self.n_samples, self.n_features = X.shape
        self.weights = np.zeros(self.n_features)
        self.bias = 0

        for _ in range(self.n_iterations):
            self._update_weights()

    def _update_weights(self):
        for i in range(self.n_samples):
            y_pred = self.predict(self.X[i])
            update = self.learning_rate * (self.y[i] - y_pred)
            self.weights += update * self.X[i]
            self.bias += update

    def predict(self, X):
        return np.dot(X, self.weights) + self.bias

    def score(self, X, y):
        predictions = np.where(self.predict(X) >= 0, 1, -1)
        accuracy = np.mean(predictions == y)
        return accuracy
if __name__ == "__main__":
    X,y=parseData("handwriting_alzheimers.csv")
    X = X[:, 1:]
    y = np.where(y == "P", 1, y)
    y = np.where(y == "H", -1, y)
    X = normalize(X)
    X, y = shuffle(X, y)
    X, y = shuffle(X, y)
    X, y = shuffle(X, y)
    
    # Convert y to float type
    y = np.where(y == "P", 1, -1)

    # Insert a column of ones (bias term) to X
    X = np.insert(X, 0, 1, axis=1)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Initialize and train the model
    model = Perceptron(learning_rate=0.001, n_iterations=10000)
    model.fit(X_train, y_train)

    # Calculate accuracy on the test set
    accuracy = model.score(X_test, y_test)

    print(f"Accuracy: {accuracy}")

Accuracy: 1.0
